In [1]:
# Imports
from subsbml import System, createSubsystem, combineSystems, createNewSubsystem, createBasicSubsystem, SimpleModel, SimpleReaction
import numpy as np
import pylab as plt

from bokeh.layouts import row
from bokeh.io import export_png

import warnings
import libsbml
import bokeh.io
import bokeh.plotting

In [3]:
ss1 = createSubsystem ("SBML_Models/BiSwitch_CRN.xml")
ss2 = createSubsystem ("SBML_Models/external_environment.xml")

# Create a simple rR1 membrane 
mb2 = createSubsystem("SBML_Models/rR1_membrane1_detailed.xml", membrane = True)
mb3 = createSubsystem("SBML_Models/simple_DFHBI_Bistable.xml", membrane = True)

#cell_2B = System("cell_2B")
#cell_2B.setInternal([ss1])
#cell_2B.setExternal([ss2])
#cell_2B.setMembrane(mb2,mb3)

cell_2B =  System('cell_2B',ListOfInternalSubsystems = [ss1],
                 ListOfExternalSubsystems = [ss2],
                 ListOfMembraneSubsystems = [mb2,mb3])

cell_2B_model = cell_2B.getModel()
cell_2B_model.renameCompartments('cell_2B_external', 'actual_external')

##### Set Species Amount #####

cell_2B_model.setSpeciesAmount('rna_rR1_e', 0, compartment = 'actual_external')
cell_2B_model.setSpeciesAmount('DFHBI_e', 100, compartment = 'actual_external')

cell_2B_model.setSpeciesAmount('Core1_OFF', 1.2, compartment = 'cell_2B_internal')
cell_2B_model.setSpeciesAmount('Core2_OFF', 0.99, compartment = 'cell_2B_internal')
cell_2B_model.setSpeciesAmount('dna_dA2', 1.2, compartment = 'cell_2B_internal')
cell_2B_model.setSpeciesAmount('dna_dA1', 1.2, compartment = 'cell_2B_internal')

#cell_2B_model.setSpeciesAmount('rna_rR2', 0.5, compartment = 'cell_2B_internal')

cell_2B_model.setSpeciesAmount('protein_RNAseH', 0.0044, compartment = 'cell_2B_internal') #0.00333
#cell_2B_model.setSpeciesAmount('protein_RNAseH', 40, compartment = 'cell_2B_internal')

cell_2B_model.setSpeciesAmount('protein_RNAP', 0.030, compartment = 'cell_2B_internal') #0.025

cell_2B_model.getSBMLDocument().getModel().getCompartment(1).setSize(1e-4) 

cell_2B_model.writeSBML('SBML_Models/cell_2B_model.xml')

The subsystem from SBML_Models/rR1_membrane1_detailed.xml has multiple compartments
The subsystem from SBML_Models/simple_DFHBI_Bistable.xml has multiple compartments


1

In [11]:
# Calling Names

X_id1 = cell_2B_model.getSpeciesByName('complex_Core1_ON').getId()
X_id2 = cell_2B_model.getSpeciesByName('Core1_OFF').getId()
X_id3 = cell_2B_model.getSpeciesByName('complex_Core2_ON').getId()
X_id4 = cell_2B_model.getSpeciesByName('Core2_OFF').getId()

X_id5 = cell_2B_model.getSpeciesByName("rna_rR2").getId()
X_id6 = cell_2B_model.getSpeciesByName("rna_rR1", compartment = 'cell_2B_internal').getId()
X_id7 = cell_2B_model.getSpeciesByName("rna_rR1_e", compartment = 'actual_external').getId()

X_id8 = cell_2B_model.getSpeciesByName("complex_Core2_AI").getId()
X_id9 = cell_2B_model.getSpeciesByName("complex_Core1_AI").getId()

X_id10 = cell_2B_model.getSpeciesByName("dna_dA2").getId()
X_id11 = cell_2B_model.getSpeciesByName("dna_dA1").getId()

X_id12 = cell_2B_model.getSpeciesByName("complex_Core1_ON_protein_RNAP").getId()
X_id13 = cell_2B_model.getSpeciesByName("complex_Core2_ON_protein_RNAP").getId()

X_id14 = cell_2B_model.getSpeciesByName("complex_Core1_OFF_protein_RNAP").getId()
X_id15 = cell_2B_model.getSpeciesByName("complex_Core2_OFF_protein_RNAP").getId()

S_id1 = cell_2B_model.getSpeciesByName("DFHBI_B", compartment = "cell_2B_internal").getId()
print(S_id1)

DFHBI_B


In [12]:
# Simulate with BioScrape
timepoints = np.linspace(0, 28800, 1000)
R,_ = cell_2B_model.simulateWithBioscrape(timepoints)
timepoints = timepoints/3600

# For label convenience
x = 'Time (hours)'
y = 'Concentration (uM)'

bokeh.io.output_notebook()
a = bokeh.plotting.figure(plot_width=400, plot_height=300, x_axis_label = x, y_axis_label=y)
a.circle(timepoints, R[X_id1], legend_label = "Core1 ON" , color = "blue")
a.circle(timepoints, R[X_id3], legend_label = "Core2 ON", color = "red")
a.legend.click_policy="hide"
a.legend.location="center_right"

b = bokeh.plotting.figure(plot_width=400, plot_height=300, x_axis_label = x, y_axis_label=y)
b.circle(timepoints, R[X_id6], legend_label = "rR1_i", color = "blue")
#b.circle(timepoints, R[S_id1], legend_label = "rR1_i", color = "blue")
b.legend.click_policy="hide"
b.legend.location="center_right"

c = bokeh.plotting.figure(plot_width=400, plot_height=300, x_axis_label = x, y_axis_label=y)
c.circle(timepoints, R[X_id7], legend_label = "rR1_e", color = "orange")
c.legend.click_policy="hide"
c.legend.location="center_right"

d = bokeh.plotting.figure(plot_width=400, plot_height=300, x_axis_label = x, y_axis_label=y)
d.circle(timepoints, R[X_id5], legend_label = "rR2" , color = "red")
d.legend.click_policy="hide"
d.legend.location="center_right"

bokeh.io.show(row(a, c, b, d))

Loading BokehJS ...

In [13]:
e = bokeh.plotting.figure(plot_width=400, plot_height=300, x_axis_label = x, y_axis_label=y)
e.circle(timepoints, R[X_id8], legend_label = "dA1:rR1" , color = "red")
e.circle(timepoints, R[X_id9], legend_label = "dA2:rR2", color = "blue")
e.legend.click_policy="hide"
e.legend.location="center_right"

f = bokeh.plotting.figure(plot_width=400, plot_height=300, x_axis_label = x, y_axis_label=y)
f.circle(timepoints, R[S_id1], legend_label = "DFHBI", color = "green")
f.legend.click_policy="hide"
f.legend.location="center_right"

bokeh.io.show(row(e, f))

## Complex Core 1 On initially ##

In [ ]:


cell_2B_model = cell_2B.getModel()

##### Set Species Amount #####

cell_2B_model.setSpeciesAmount('rna_rR1_e', 0, compartment = 'cell_2B_external')
#cell_2_model.setSpeciesAmount('rna_rR1', 100, compartment = 'cell_2B_external')

cell_2B_model.setSpeciesAmount('complex_Core1_ON', 0.1, compartment = 'cell_2B_internal')
cell_2B_model.setSpeciesAmount('Core2_OFF', 0.1, compartment = 'cell_2B_internal')
cell_2B_model.setSpeciesAmount('dna_dA2', 0.05, compartment = 'cell_2B_internal')
cell_2B_model.setSpeciesAmount('dna_dA1', 0.1, compartment = 'cell_2B_internal')
#cell_2B_model.setSpeciesAmount('rna_rR1', 0.5, compartment = 'cell_2B_internal')

cell_2B_model.setSpeciesAmount('protein_RNAseH', 0.0044, compartment = 'cell_2B_internal') #0.00333
#cell_2B_model.setSpeciesAmount('protein_RNAseH', 40, compartment = 'cell_2B_internal')

cell_2B_model.setSpeciesAmount('protein_RNAP', 0.030, compartment = 'cell_2B_internal') #0.025

#cell_2B_model.getSBMLDocument().getModel().getCompartment(1).setSize(1e-4) 

cell_2B_model.writeSBML('SBML_Models/cell_2B_model.xml')

In [ ]:
# Simulate with BioScrape
timepoints = np.linspace(0, 86400, 1000)
P,_ = cell_2B_model.simulateWithBioscrape(timepoints)
timepoints = timepoints/3600

# For label convenience
x = 'Time (hours)'
y = 'Concentration (uM)'

bokeh.io.output_notebook()
a = bokeh.plotting.figure(plot_width=400, plot_height=300, x_axis_label = x, y_axis_label=y)
a.circle(timepoints, P[X_id1], legend_label = "Core1 ON" , color = "blue")
a.circle(timepoints, P[X_id3], legend_label = "Core2 ON", color = "red")
a.legend.click_policy="hide"
a.legend.location="center_right"

b = bokeh.plotting.figure(plot_width=400, plot_height=300, x_axis_label = x, y_axis_label=y)
b.circle(timepoints, P[X_id6], legend_label = "rR1_i", color = "blue")
b.legend.click_policy="hide"
b.legend.location="center_right"

c = bokeh.plotting.figure(plot_width=400, plot_height=300, x_axis_label = x, y_axis_label=y)
c.circle(timepoints, P[X_id7], legend_label = "rR1_e", color = "orange")
c.legend.click_policy="hide"
c.legend.location="center_right"

d = bokeh.plotting.figure(plot_width=400, plot_height=300, x_axis_label = x, y_axis_label=y)
d.circle(timepoints, P[X_id5], legend_label = "rR2" , color = "red")
d.legend.click_policy="hide"
d.legend.location="center_right"

bokeh.io.show(row(a, c, b, d))

In [ ]:
e = bokeh.plotting.figure(plot_width=400, plot_height=300, x_axis_label = x, y_axis_label=y)
e.circle(timepoints, P[X_id14], legend_label = "RNAP 1" , color = "blue")
e.circle(timepoints, P[X_id15], legend_label = "RNAP 2" , color = "red")

e.legend.click_policy="hide"
e.legend.location="center_right"

bokeh.io.show(row(e))